In [5]:
%pip install requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl (198 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.6.15-py3-none-any.whl (157 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
df =pd.read_csv("events.csv")
df

,Event_ID,Event_Name,Event_Date,Outcome,Primary_Type,Secondary_Type,Region,Country_1,Country_2,External_Actors,Key_Individuals,Key_Groups_Entities,Date_Justification,Search_Terms
0,EUR_2022_UKRAINE_CONFLICT,Full-Scale Invasion of Ukraine,2022-02-24,Conflict,Invasion,Interstate War,Europe,Ukraine,Russia,"Belarus,Chinese Hackers,International Criminal...","Putin,Zelenskyy,Biden,Alexander Bortnikov,Niko...","Kremlin,NATO,Russian Army,Ukrainian Army,Donet...",NaN,"Ukraine invasion,Russia special military opera..."
1,MEA_2023_GAZA_CONFLICT,Hamas Attack on Israel,2023-10-07,Conflict,Cross-border Attack,Asymmetric War,Middle East,Israel,Palestine,"Iran,IDF,Gaza,Islamic Jihad","Netanyahu,Sinwar,Mohammed Deif","Hamas,Israel Defense Forces,Al-Qassam Brigades...",NaN,"Hamas attack,Iron Swords,Gaza invasion"
2,AFR_2020_TIGRAY_CONFLICT,Tigray War Start,2020-11-04,Conflict,Civil War,Intervention,Africa,Ethiopia,NaN,"Eritrea,militas from Amhara and Afar","Abiy Ahmed,Debretsion Gebremichael,Getachew Re...","Tigray,TDF,ENDF,Eritrean Defence Forces,TPLF,IRAT",NaN,"Tigray war,Ethiopia conflict,Abiy Ahmed"
3,AFR_2011_LIBYA_CONFLICT,Libyan Civil War Start,2011-02-15,Conflict,Civil War,Foreign Intervention,Africa,Libya,NaN,"NATO,Turkey,UAE,Egypt,France,Wagner Group","Gaddafi,Mustafa Abdul Jalil,Khalifa Haftar,Fay...","National Transitional Council,NTC,UNSC Resolut...",NaN,"Libya civil war,Gaddafi speech,Benghazi uprising"
4,EUR_2008_GEORGIA_CONFLICT,Russo-Georgian War,2008-08-07,Conflict,Invasion,Interstate War,Europe,Georgia,Russia,"USA,NATO,France,Separatist authorities and arm...","Saakashvili,Medvedev,Putin,Borisov,Nicolas Sar...","South Ossetia,Abkhazia,Georgian Armed forces,R...",NaN,"Russo-Georgian war,South Ossetia conflict"
5,MEA_2015_YEMEN_CONFLICT,Saudi-led Intervention in Yemen,2015-03-26,Conflict,Intervention,Proxy War,Middle East,Yemen,Saudi Arabia,"United Arab Emirates, Egypt, Jordan, Morocco, ...","Hadi,King Salman,Mohammed bin Salman,President...","Houthi,Operation Decisive Storm,Saudi-led coal...",NaN,"Saudi intervention Yemen,Houthi conflict"
6,MEA_2011_SYRIA_CONFLICT,Syrian Civil War Begins,2011-03-15,Conflict,Civil War,Proxy War,Middle East,Syria,Saudi Arabia,"Russia,USA,Iran,Turkey,Saudi Arabia,Qatar","Assad,Obama,Abu Mohammed al-Jolani,Putin,Qasse...","Free Syrian Army,FSA,Daraa protests,ISIS,Wagne...",NaN,"Syria civil war,Assad speech,Daraa"
7,EUR_2020_NAGORNO_CONFLICT,Second Nagorno-Karabakh War,2020-09-27,Conflict,Interstate War,NaN,Europe,Armenia,Azerbaijan,"Russia,Turkey,Israel,Syrian mercenaries,Afghan...","Pashinyan,Aliyev,Putin","Nagorno-Karabakh,Artsakh,Armenian Army,Azerbai...",NaN,"Nagorno-Karabakh war,Armenia Azerbaijan conflict"
8,EUR_2014_DONBAS_CONFLICT,Donbas Conflict Start,2014-04-06,Conflict,Hybrid War,Insurgency,Europe,Ukraine,Russia,"OSCE,France,Germany,USA,EU","Turchynov,Strelkov,Viktor Yanukovych,Petro Por...","Donetsk People's Republic,Luhansk People's Rep...",NaN,"Donbas war,Ukraine separatists"
9,EUR_2014_CRIMEA_CONFLICT,Annexation of Crimea,2014-02-27,Conflict,Annexation,Hybrid War,Europe,Ukraine,Russia,"US,EU,NATO,UN,CIS,OSCE","Aksyonov,Putin,Oleksandr Turchynov,Viktor Yanu...","little green men,referendum,Supreme Council of...",NaN,"Crimea annexation,Russia Crimea"


# NEWS DATA COLLECTION

In [ ]:
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import dotenv
dotenv.load_dotenv()

# --- CONFIGURATION & CONSTANTS ---
LEAD_UP_DAYS = 90
DATA_DIR = "raw_data"
EVENTS_FILE = "events.csv"

# --- API HELPERS ---

def build_query(event):
    """Builds a robust boolean query for the APIs."""
    
    # 1. Country terms (must have at least one)
    country_terms = f'({event["Country_1"]}'
    if pd.notna(event["Country_2"]):
        country_terms += f' OR {event["Country_2"]}'
    country_terms += ')'

    # 2. Key entities (should have at least one)
    actors = []
    if pd.notna(event["Key_Individuals"]):
        actors.extend([f'{name.strip()}' for name in event["Key_Individuals"].split(',')])
    if  pd.notna(event["Key_Groups_Entities"]):
         actors.extend([f'{name.strip()}' for name in event["Key_Groups_Entities"].split(',')])
    
    actor_terms = " AND (" + " OR ".join(actors) + ")" if actors else ""

    # 3. Thematic terms (from the Search_Terms column)
    theme_terms = ""
    if  pd.notna(event["Search_Terms"]):
        themes = [f'{term.strip()}' for term in event["Search_Terms"].split(',')]
        theme_terms = " AND (" + " OR ".join(themes) + ")"

    # Combine all parts
    full_query = country_terms + actor_terms + theme_terms
    return full_query

def fetch_guardian_data(query, start_date, end_date):
    """Fetches data from The Guardian API, handling pagination."""
    print("  -> Fetching from The Guardian...")
    articles = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        params = {
            'q': query,
            'from-date': start_date,
            'to-date': end_date,
            'api-key': os.getenv('GUARDIAN_API_KEY'),
            'page-size': 50,
            'page': page,
            'show-fields': 'bodyText'
        }
        try:
            response = requests.get('https://content.guardianapis.com/search', params=params)
            response.raise_for_status()
            data = response.json()['response']
            
            for item in data.get('results', []):
                articles.append({
                    'source': 'The Guardian',
                    'date': item['webPublicationDate'],
                    'headline': item['webTitle'],
                    'snippet': item.get('fields', {}).get('bodyText', '')[:500] # Get first 500 chars of body - why?
                })
            
            total_pages = data.get('pages', 1)
            print(f"     Got page {page}/{total_pages}...")
            page += 1
            time.sleep(1) # IMPORTANT: Respect API rate limits
        except requests.exceptions.RequestException as e:
            print(f"     Error fetching from The Guardian: {e}")
            break
            
    return articles

def build_nyt_query_and_filter(event):
    """
    Builds a more flexible query and a separate filter for the NYT.
    Returns a tuple: (query, filter_query)
    """
    # 1. The Filter Query (fq): Highly reliable for location
    # We use the 'glocations' field in the NYT data.
    glocations = []
    if pd.notna(event["Country_1"]):
        glocations.append(f'"{event["Country_1"].upper()}"')
    if pd.notna(event["Country_2"]):
        glocations.append(f'"{event["Country_2"].upper()}"')
    filter_query = f'glocations:({" OR ".join(glocations)})' if glocations else None

    # 2. The Keyword Query (q): A flexible search for actors and themes
    keywords = []
    if pd.notna(event["Search_Terms"]):
        keywords.extend([f'"{term.strip()}"' for term in event["Search_Terms"].split(',')])
    if pd.notna(event["Key_Groups_Entities"]):
        # Add the first 2 key groups for relevance
        keywords.extend([f'"{term.strip()}"' for term in event["Key_Groups_Entities"].split(',')[:2]])
        
    keyword_query = " OR ".join(keywords) if keywords else event['Event_Name'] # Fallback to event name
    
    return keyword_query, filter_query



In [39]:
event_date = datetime.strptime(df.loc[0]['Event_Date'], '%Y-%m-%d')
start_datetime = event_date - timedelta(days=LEAD_UP_DAYS)

start_date_str = start_datetime.strftime('%Y-%m-%d')
end_date_str = event_date.strftime('%Y-%m-%d')
articles = fetch_guardian_data(build_query(df.loc[0]), start_date_str, end_date_str)
articles

  -> Fetching from The Guardian...
     Got page 1/17...
     Got page 2/17...
     Got page 3/17...
     Got page 4/17...
     Got page 5/17...
     Got page 6/17...
     Got page 7/17...
     Got page 8/17...
     Got page 9/17...
     Got page 10/17...
     Got page 11/17...
     Got page 12/17...
     Got page 13/17...
     Got page 14/17...
     Got page 15/17...
     Got page 16/17...
     Got page 17/17...


[{'source': 'The Guardian',
  'date': '2022-02-24T05:57:05Z',
  'headline': 'Markets shaken after Putin announces special military operation – as it happened',
  'snippet': 'This blog is closing now but please continue to follow our live coverage on our new liveblog here. Thank you for reading. The Russian rouble has fallen 5.4% to a record low on Thursday, after financial markets were shaken by Vladimir Putin’s decision to invade Ukraine. The Moscow Exchange said on Thursday morning it had suspended trading on all markets. Brent crude oil prices crossed $100/barrel for first time since 2014. Safe haven currencies such as the yen and US dollar rose, along with gold '},
 {'source': 'The Guardian',
  'date': '2022-02-23T05:41:54Z',
  'headline': 'Ukraine president calls up reservists as sanctions coalition against Russia grows ',
  'snippet': 'Ukraine’s president has called up the country’s reservists and warned that Ukraine could face a battle for its very existence as more countries jo

In [67]:
len(articles)

836

In [65]:
start_date_nyt = start_datetime.strftime('%Y%m%d')
end_date_nyt = event_date.strftime('%Y%m%d')
data =fetch_nyt_data(build_nyt_query_and_filter(df.loc[0]), start_date_nyt, end_date_nyt)

  -> Fetching from The New York Times...


In [59]:
params = {
'q': build_nyt_query(df.loc[0]),
'begin_date': start_date_nyt,
'end_date': end_date_nyt,
'api-key': os.getenv('NYT_API_KEY'),
'page': 0
}
response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params=params)
data = response.json()['response']
docs = data.get('docs', [])
docs

[{'abstract': 'If the West fails to meet its security demands, Moscow could take measures like placing nuclear missiles close to the U.S. coastline, Russian officials have hinted.',
  'byline': {'original': 'By Anton Troianovski and David E. Sanger'},
  'document_type': 'article',
  'headline': {'main': 'Russia Issues Subtle Threats More Far-Reaching Than a Ukraine Invasion',
   'kicker': '',
   'print_headline': 'Putin Could Cause Trouble Not Just in Ukraine but in West, Too'},
  '_id': 'nyt://article/73e801a9-d932-5899-96ec-11d092bbb45e',
  'keywords': [{'name': 'Location', 'value': 'Russia', 'rank': 1},
   {'name': 'Location', 'value': 'Ukraine', 'rank': 2},
   {'name': 'Subject', 'value': 'Defense and Military Forces', 'rank': 3},
   {'name': 'Subject', 'value': 'Cyberwarfare and Defense', 'rank': 4},
   {'name': 'Organization',
    'value': 'North Atlantic Treaty Organization',
    'rank': 5},
   {'name': 'Subject',
    'value': 'United States International Relations',
    'rank':

In [57]:
len(docs)

10

In [ ]:
"""Main function to orchestrate the data acquisition process."""
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

events_df = pd.read_csv(EVENTS_FILE)

for _, event in events_df.iterrows():
    event_id = event['Event_ID']
    output_filename = os.path.join(DATA_DIR, f"{event_id}.csv")

    if os.path.exists(output_filename):
        print(f"Skipping {event_id}: Data file already exists.")
        continue

    print(f"\nProcessing Event: {event_id} - {event['Event_Name']}")

    # Prepare dates and query
    event_date = datetime.strptime(event['Event_Date'], '%Y-%m-%d')
    start_datetime = event_date - timedelta(days=LEAD_UP_DAYS)
    
    start_date_str = start_datetime.strftime('%Y-%m-%d')
    end_date_str = event_date.strftime('%Y-%m-%d')
    
    query = build_query(event)
    print(f"  -> Query: {query[:200]}...") # Print first 200 chars of query

    # Fetch data from all sources
    guardian_articles = fetch_guardian_data(event, query, start_date_str, end_date_str)

    # Combine and save
    all_articles = guardian_articles 
    if all_articles:
        df = pd.DataFrame(all_articles)
        df.to_csv(output_filename, index=False, encoding='utf-8')
        print(f"Success! Saved {len(df)} articles to {output_filename}")
    else:
        print(f"Warning: No articles found for {event_id}. An empty file will not be created.")
